In [1]:
import torch
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import numpy as np
from models import *
from dataset import MyCelebA

In [2]:
path = "logs/VanillaVAE/version_0/checkpoints/last.ckpt"

model = VanillaVAE(3, 512)
file = torch.load(path)
parsed_file = {k.replace('model.', ''): v for k, v in file['state_dict'].items()}
model.load_state_dict(parsed_file)

<All keys matched successfully>

In [3]:
import pandas as pd

attrs = pd.read_csv('data/celeba/list_attr_celeba.txt', sep='\s+', header=1, index_col=0)
attrs.columns[15]

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Josh\AppData\Local\Temp\ipykernel_18028\1413202656.py:3: SyntaxWarning: invalid escape sequence '\s'
  attrs = pd.read_csv('data/celeba/list_attr_celeba.txt', sep='\s+', header=1, index_col=0)


'Eyeglasses'

In [ ]:
indices = pd.Series(attrs[attrs.Eyeglasses == 1][:2000].index)
indices = list(indices.str.replace('.jpg', '').astype(int) - 1)
not_sunglas_indices = pd.Series(attrs[attrs.Eyeglasses == -1][:2000].index)
not_sunglas_indices = list(not_sunglas_indices.str.replace('.jpg', '').astype(int) - 1)

In [6]:
from torchvision import transforms
tf = transforms.Compose([transforms.CenterCrop(148),
                        transforms.Resize(64),
                        transforms.ToTensor(),])

sunglasses_idx = torch.tensor(attrs.columns[15] == attrs.columns, dtype = torch.int16)
not_sunglasses_idx = torch.tensor(attrs.columns[15] != attrs.columns, dtype = torch.int16)
celeba_images = MyCelebA(root='data', target_type='attr', transform=tf)

In [7]:
sunglasses_celeba_images = torch.utils.data.Subset(celeba_images, indices)
not_sunglasses_celeba_images = torch.utils.data.Subset(celeba_images, not_sunglas_indices)

In [12]:
# Load the datasets
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = model.to('cuda')  # Move to GPU if available
model.eval()  # Set the model to evaluation mode

sunglasses_loader = DataLoader(sunglasses_celeba_images, batch_size=100, pin_memory=True)
not_sunglasses_loader = DataLoader(not_sunglasses_celeba_images, batch_size=100, pin_memory=True)

# Prepare the data
def extract_latent_features(loader, model):
    features = []
    for batch in loader:
        images, _ = batch
        images = images.to('cuda')  # Move to GPU if available
        with torch.no_grad():
            # Pass through encoder and reparameterization
            mu, log_var = model.encode(images)
            z = model.reparameterize(mu, log_var)
        features.append(z.cpu().numpy())
    return np.vstack(features)

sunglasses_features = extract_latent_features(sunglasses_loader, model)
not_sunglasses_features = extract_latent_features(not_sunglasses_loader, model)

# Combine features and labels
X = np.vstack([sunglasses_features, not_sunglasses_features])
y = np.hstack([np.ones(len(sunglasses_features)), np.zeros(len(not_sunglasses_features))])

# Train the SVM
svm = LinearSVC()
svm.fit(X, y)

# Evaluate the SVM
y_pred = svm.predict(X)
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)

print(f'SVM Accuracy: {accuracy * 100:.2f}%')
print(f'SVM Precision: {precision * 100:.2f}%')
print(f'SVM Recall: {recall * 100:.2f}%')
print(f'SVM F1 Score: {f1 * 100:.2f}%')





SVM Accuracy: 76.11%
SVM Precision: 76.80%
SVM Recall: 74.84%
SVM F1 Score: 75.81%


In [9]:
from IPython import display
from PIL import Image

# display(None)

im = celeba_images[0][0].numpy()

Image.fromarray(im[0]*255).show()

input = torch.tensor(im[np.newaxis, :, :])
latent = model.encode(input.to('cuda'))

latent = model.reparameterize(latent[0], latent[1]).cpu().detach().numpy()
latent += svm.coef_ * 8

final_image = model.decode(torch.tensor(latent).to('cuda')).cpu().detach().numpy()
Image.fromarray(final_image[0,0] * 255).show()
# sunglasses_im = im + svm.coef_[0][0] * 0.1
# Image.fromarray(sunglasses_im*255).show()


In [11]:
from torchvision import utils as vutils

test_input, test_label = next(iter(not_sunglasses_loader))

model.eval()
vutils.save_image(test_input.data,
                    "originals.png",
                    normalize=True,
                    nrow=10)
#         test_input, test_label = batch
latent = model.encode(test_input.to("cuda"))
latent = model.reparameterize(latent[0], latent[1]).detach().cpu().numpy()

latent = (latent + svm.coef_ *(7 - latent @ svm.coef_.T / np.linalg.norm(svm.coef_.T))).astype(np.float32)
recons = model.decode(torch.tensor(latent).to("cuda")).cpu().detach()

vutils.save_image(recons.data,
                    "sunglasses.png",
                    normalize=True,
                    nrow=10)